## Yolov10

### 3/10/24
Voy a probar yolov10 de ultralytics.
Mire este vide https://youtube.com/watch?v=PfQwNe0P-G4&t=1320s&ab_channel=Computervisionengineer
https://www.labelvisor.com/detailed-architecture-of-yolov10/

In [1]:
import os
from ultralytics import YOLO
import pandas as pd
import warnings

In [2]:
warnings.filterwarnings("ignore")  # Ignore DeprecationWarnings only

In [3]:
# Load a pre-trained YOLOv10n model
from ultralytics import YOLO

model = YOLO("yolov10n.pt")
# Perform object detection on an image
#results = model("bus.jpg")
# Display the results
#results[0].show()

In [6]:
image_files = "dataset_fruits_vegs/test/images/"

#### load pretrained and infer

In [7]:
import pandas as pd
import os

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Loop over each image file
for image_file in image_files:
    image_path = os.path.join(test_dir, image_file)
    
    # Get results for the image
    results = model(image_path)
    
    # Iterate over each result (if results is a list of Results objects)
    for result in results:
        # Check if the result has detected boxes
        if result.boxes is not None:
            # Extract the bounding boxes in xywh format (x, y, width, height)
            boxes = result.boxes.xywhn  # Normalized bbox in xywh format
            
            # Extract confidence and class information (if available)
            confidences = result.boxes.conf  # Confidence scores for each box
            classes = result.boxes.cls       # Class indices for each box

            class_names = [result.names[int(cls)] for cls in classes]
            
            # Combine the boxes, confidence, and class information into a single array
            combined_data = pd.DataFrame({
                'x': boxes[:, 0],
                'y': boxes[:, 1],
                'w': boxes[:, 2],
                'h': boxes[:, 3],
                'confidence': confidences,
                'class': classes,
                'name': class_names 
            })
            
            # Add the image file name to the result
            combined_data['image'] = image_file
            
            # Concatenate the result with the overall DataFrame
            all_results = pd.concat([all_results, combined_data], ignore_index=True)

# Save the concatenated results to a CSV file
all_results.to_csv("2_yolo10_inference_results.csv", index=False)

print("Inference results saved to 'yolo10_inference_results.csv'")

NameError: name 'test_dir' is not defined

In [ ]:
all_results.head()

#### Compare annotations against inference results
* Map class_id from annotations to class_name. how? find list in kaggle, if not, map manually
* Create dataframe with anotations? 
* Maybe I can concat them and plot results.
1) Check if class was predicted ok
2) Check those with confidence about certain threshold
3) Check difference between x and ys predicted vs reality

##### Parse id manually
1) Read each .txt file in the test_dir
2) Extract the class_id (the first number in the annotation file).
3) Parse the class name from the corresponding filename.
4) Map the class_id to the class name

In [ ]:
def extract_class_name(filename):
    return filename.split("_")[2]

def read_class_id(annotation_path):
    with open(annotation_path, 'r') as file:
        first_line = file.readline().strip()
        class_id = first_line.split()[0] #cos the first nuumber is the id
        return int(class_id)

In [ ]:
annotation_files = [f for f in os.listdir(test_dir) if f.endswith(".txt")]

In [ ]:
df_annotations = pd.read_csv("all_annotations_with_class_names_and_filenames.csv")

In [ ]:
df_annotations.head()

In [ ]:
df_ann

#### Comparison

In [11]:
def strip_extension(filename):
    return os.path.splitext(filename)[0]

# Add a column without extensions to both DataFrames
all_results['base_filename'] = all_results['image'].apply(strip_extension)
df_annotations['base_filename'] = df_annotations['filename'].apply(strip_extension)

merged_df = pd.merge(all_results, df_annotations, on='base_filename', how='inner', suffixes=('_pred', '_anno'))

In [12]:
merged_df.to_csv("yolov10_merged_preds_annotations.csv", index=False)

In [13]:
merged_df.head()

,x_pred,y_pred,w,h,confidence,class,name,image,base_filename,class_id,x_anno,y_anno,width,height,class_name,filename
0,0.471219,0.519214,0.691867,0.346951,0.873932,46.0,banana,208_5_banana_wob_7.jpg,208_5_banana_wob_7,1,0.463624,0.518105,0.700728,0.343750,banana,208_5_banana_wob_7.txt
1,0.276985,0.525569,0.279096,0.492139,0.449253,46.0,banana,273_3_chilli_wob_46.jpg,273_3_chilli_wob_46,13,0.340030,0.514220,0.383185,0.545966,chilli,273_3_chilli_wob_46.txt
2,0.750785,0.674841,0.498215,0.344104,0.559812,14.0,bird,400_4_tomato_wb_37.jpg,400_4_tomato_wb_37,8,0.751653,0.670883,0.470899,0.336310,tomato,400_4_tomato_wb_37.txt
3,0.505947,0.602331,0.977427,0.790007,0.388267,28.0,suitcase,400_4_tomato_wb_37.jpg,400_4_tomato_wb_37,8,0.751653,0.670883,0.470899,0.336310,tomato,400_4_tomato_wb_37.txt
4,0.348076,0.443880,0.489531,0.546773,0.320207,14.0,bird,413_0_chilli_wb_25.jpg,413_0_chilli_wb_25,12,0.354167,0.396081,0.460648,0.446677,chilli,413_0_chilli_wb_25.txt


In [14]:
merged_df.columns

Index(['x_pred', 'y_pred', 'w', 'h', 'confidence', 'class', 'name', 'image',
       'base_filename', 'class_id', 'x_anno', 'y_anno', 'width', 'height',
       'class_name', 'filename'],
      dtype='object')

### Evaluation

In [16]:
import numpy as np

In [17]:
def calculate_iou(pred_box, gt_box):
    x_pred, y_pred, w_pred, h_pred = pred_box
    x_gt, y_gt, w_gt, h_gt = gt_box

    # convert from center to corner coordinates
    pred_xmin = x_pred - w_pred / 2 # top left corner
    pred_ymin = y_pred - h_pred / 2 # top left corner
    pred_xmax = x_pred + w_pred / 2 # bottom right corner
    pred_ymax = y_pred + h_pred / 2 # bottom right corner

    # same for labeled data
    gt_xmin = x_gt - w_gt / 2
    gt_ymin = y_gt - h_gt / 2
    gt_xmax = x_gt + w_gt / 2
    gt_ymax = y_gt + h_gt / 2

    # calculate intersection
    inter_xmin = max(pred_xmin, gt_xmin)
    inter_ymin = max(pred_ymin, gt_ymin)
    inter_xmax = min(pred_xmax, gt_xmax)
    inter_ymax = min(pred_ymax, gt_ymax)

    inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)
    pred_area = (pred_xmax - pred_xmin) * (pred_ymax - pred_ymin)
    gt_area = (gt_xmax - gt_xmin) * (gt_ymax - gt_ymin)
    
    union_area = pred_area + gt_area - inter_area
    
    iou = inter_area / union_area if union_area > 0 else 0
    return iou

In [27]:
def calculate_iou(x1, y1, w1, h1, x2, y2, w2, h2):
    # Convert from center x, center y, width, height to xmin, ymin, xmax, ymax
    xmin1 = x1 - w1 / 2
    ymin1 = y1 - h1 / 2
    xmax1 = x1 + w1 / 2
    ymax1 = y1 + h1 / 2

    xmin2 = x2 - w2 / 2
    ymin2 = y2 - h2 / 2
    xmax2 = x2 + w2 / 2
    ymax2 = y2 + h2 / 2

    # Calculate the intersection area
    inter_xmin = max(xmin1, xmin2)
    inter_ymin = max(ymin1, ymin2)
    inter_xmax = min(xmax1, xmax2)
    inter_ymax = min(ymax1, ymax2)

    inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)

    # Calculate the area of each bounding box
    box1_area = (xmax1 - xmin1) * (ymax1 - ymin1)
    box2_area = (xmax2 - xmin2) * (ymax2 - ymin2)

    # Calculate the union area
    union_area = box1_area + box2_area - inter_area

    # Calculate IoU
    iou = inter_area / union_area if union_area > 0 else 0

    return iou

In [30]:
def evaluate_object_detection(df, iou_threshold=0.5):
    results = {
        'class_metrics': {},
        'mAP': 0
    }

    # Initialize metrics for all unique class names
    unique_classes = df['class_name'].unique()
    for cls in unique_classes:
        results['class_metrics'][cls] = {'tp': 0, 'fp': 0, 'fn': 0, 'precision': 0, 'recall': 0}

    # Iterate through the dataframe to evaluate predictions
    for index, row in df.iterrows():
        pred_class = row['name']  # Predicted class name
        true_class = row['class_name']  # Ground truth class name

        if pred_class == true_class:
            # Calculate IoU
            iou = calculate_iou(
                row['x_pred'], row['y_pred'], row['w'], row['h'],
                row['x_anno'], row['y_anno'], row['width'], row['height']
            )

            if iou >= iou_threshold:
                # True positive
                results['class_metrics'][true_class]['tp'] += 1
            else:
                # False negative (missed detection of the true class)
                results['class_metrics'][true_class]['fn'] += 1
        else:
            # False positive (wrong class prediction)
            if pred_class in results['class_metrics']:
                results['class_metrics'][pred_class]['fp'] += 1
            else:
                # If pred_class is not initialized, initialize it
                results['class_metrics'][pred_class] = {'tp': 0, 'fp': 1, 'fn': 0, 'precision': 0, 'recall': 0}

            # False negative (missed detection of the true class)
            results['class_metrics'][true_class]['fn'] += 1

    # Calculate precision and recall for each class
    for cls in results['class_metrics']:
        tp = results['class_metrics'][cls]['tp']
        fp = results['class_metrics'][cls]['fp']
        fn = results['class_metrics'][cls]['fn']
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        results['class_metrics'][cls]['precision'] = precision
        results['class_metrics'][cls]['recall'] = recall

    # Calculate mAP (mean Average Precision)
    total_precision = sum(metrics['precision'] for metrics in results['class_metrics'].values())
    num_classes = len(results['class_metrics'])
    results['mAP'] = total_precision / num_classes if num_classes > 0 else 0

    return results


In [31]:
metrics = evaluate_object_detection(merged_df)
print(metrics)

{'class_metrics': {'banana': {'tp': 75, 'fp': 8, 'fn': 114, 'precision': 0.9036144578313253, 'recall': 0.3968253968253968}, 'chilli': {'tp': 0, 'fp': 0, 'fn': 72, 'precision': 0, 'recall': 0.0}, 'tomato': {'tp': 0, 'fp': 0, 'fn': 75, 'precision': 0, 'recall': 0.0}, 'apple': {'tp': 28, 'fp': 267, 'fn': 244, 'precision': 0.09491525423728814, 'recall': 0.10294117647058823}, 'lemon': {'tp': 0, 'fp': 0, 'fn': 317, 'precision': 0, 'recall': 0.0}, 'blackberries - 9.txt': {'tp': 0, 'fp': 0, 'fn': 2, 'precision': 0, 'recall': 0.0}, 'blackberries - 1.txt': {'tp': 0, 'fp': 0, 'fn': 1, 'precision': 0, 'recall': 0.0}, 'grapes': {'tp': 0, 'fp': 0, 'fn': 47, 'precision': 0, 'recall': 0.0}, 'blackberries - 12.txt': {'tp': 0, 'fp': 0, 'fn': 3, 'precision': 0, 'recall': 0.0}, 'blackberries - 5.txt': {'tp': 0, 'fp': 0, 'fn': 2, 'precision': 0, 'recall': 0.0}, 'blackberries - 4.txt': {'tp': 0, 'fp': 0, 'fn': 4, 'precision': 0, 'recall': 0.0}, 'raspberry - 25.txt': {'tp': 0, 'fp': 0, 'fn': 4, 'precision': 

In [33]:
class_metrics = metrics['class_metrics']


In [35]:
df_metrics = pd.DataFrame.from_dict(class_metrics, orient='index')

In [36]:
df_metrics.head()

,tp,fp,fn,precision,recall
banana,75,8,114,0.903614,0.396825
chilli,0,0,72,0.000000,0.000000
tomato,0,0,75,0.000000,0.000000
apple,28,267,244,0.094915,0.102941
lemon,0,0,317,0.000000,0.000000


In [38]:
print(df_metrics)
print("Number of columns:", len(df_metrics.columns))

                       tp   fp   fn  precision    recall
banana                 75    8  114   0.903614  0.396825
chilli                  0    0   72   0.000000  0.000000
tomato                  0    0   75   0.000000  0.000000
apple                  28  267  244   0.094915  0.102941
lemon                   0    0  317   0.000000  0.000000
blackberries - 9.txt    0    0    2   0.000000  0.000000
blackberries - 1.txt    0    0    1   0.000000  0.000000
grapes                  0    0   47   0.000000  0.000000
blackberries - 12.txt   0    0    3   0.000000  0.000000
blackberries - 5.txt    0    0    2   0.000000  0.000000
blackberries - 4.txt    0    0    4   0.000000  0.000000
raspberry - 25.txt      0    0    4   0.000000  0.000000
raspberry - 21.txt      0    0    8   0.000000  0.000000
raspberry - 7.txt       0    0    2   0.000000  0.000000
raspberry - 2.txt       0    0    2   0.000000  0.000000
blackberries - 7.txt    0    0    2   0.000000  0.000000
blackberries - 20.txt   0    0 

In [39]:
new_row = [None, None, None, None, metrics['mAP']]

In [40]:
print("Length of new_row:", len(new_row))

Length of new_row: 5


In [41]:
df_metrics.loc['mAP'] = new_row

In [43]:
df_metrics

,tp,fp,fn,precision,recall
banana,75.0,8.0,114.0,0.903614,0.396825
chilli,0.0,0.0,72.0,0.000000,0.000000
tomato,0.0,0.0,75.0,0.000000,0.000000
apple,28.0,267.0,244.0,0.094915,0.102941
lemon,0.0,0.0,317.0,0.000000,0.000000
blackberries - 9.txt,0.0,0.0,2.0,0.000000,0.000000
blackberries - 1.txt,0.0,0.0,1.0,0.000000,0.000000
grapes,0.0,0.0,47.0,0.000000,0.000000
blackberries - 12.txt,0.0,0.0,3.0,0.000000,0.000000
blackberries - 5.txt,0.0,0.0,2.0,0.000000,0.000000
